In [10]:
import pandas as pd
import pyspark as spark
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [17]:
from pyspark.sql.types import *

In [5]:
df = pd.read_csv('/Users/rohitsuvarna/Downloads/finalOutTwo.csv',header=None)

In [6]:
df.head()

,0,1,2
0,2017-10-31T05:00:00.000-04:00,0.092263,6158.76
1,2017-10-31T06:00:00.000-04:00,0.103770,6105.90
2,2017-10-31T07:00:00.000-04:00,0.111691,6094.36
3,2017-10-31T08:00:00.000-04:00,0.082134,6125.13
4,2017-10-31T09:00:00.000-04:00,0.089370,6165.00


In [7]:
df.columns = ['DateTime','Sentiment_score','Price']

In [11]:
df.shape

(211, 3)

In [12]:
spark_df = sql.createDataFrame(df)

In [14]:
spark_df.show(20)

+--------------------+-------------------+-------+
|            DateTime|    Sentiment_score|  Price|
+--------------------+-------------------+-------+
|2017-10-31T05:00:...|0.09226261767339004|6158.76|
|2017-10-31T06:00:...|0.10376996608248826| 6105.9|
|2017-10-31T07:00:...|0.11169094251431187|6094.36|
|2017-10-31T08:00:...|0.08213433168664136|6125.13|
|2017-10-31T09:00:...|0.08937042968493204| 6165.0|
|2017-10-31T10:00:...| 0.1026717642680184|6170.77|
|2017-10-31T11:00:...|0.12814641170317434|6233.74|
|2017-10-31T12:00:...|0.10549754665746006|6201.03|
|2017-10-31T13:00:...|  0.138457897343971|6332.34|
|2017-10-31T14:00:...|0.11779438208175155|6363.13|
|2017-10-31T15:00:...|0.09131997304766154|6365.16|
|2017-10-31T16:00:...|0.09301943334583278|6364.78|
|2017-10-31T17:00:...|0.09206832917391784|6361.79|
|2017-10-31T18:00:...|0.10691324426615843|6335.64|
|2017-10-31T19:00:...|0.12217075088870975|6341.15|
|2017-10-31T20:00:...|0.12817142761147934|6370.08|
|2017-10-31T21:00:...|0.0923858

In [15]:
spark_df.printSchema()

root
 |-- DateTime: string (nullable = true)
 |-- Sentiment_score: double (nullable = true)
 |-- Price: double (nullable = true)



In [18]:
CleanDF = spark_df.withColumn("DateTime_casted",spark_df['DateTime'].cast(TimestampType()))



In [20]:
CleanDF.printSchema()

root
 |-- DateTime: string (nullable = true)
 |-- Sentiment_score: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- DateTime_casted: timestamp (nullable = true)



In [21]:
FinalDf = CleanDF.selectExpr("DateTime_casted as Date_Time", "Sentiment_score",'Price')



In [22]:
FinalDf.show()

+-------------------+-------------------+-------+
|          Date_Time|    Sentiment_score|  Price|
+-------------------+-------------------+-------+
|2017-10-31 05:00:00|0.09226261767339004|6158.76|
|2017-10-31 06:00:00|0.10376996608248826| 6105.9|
|2017-10-31 07:00:00|0.11169094251431187|6094.36|
|2017-10-31 08:00:00|0.08213433168664136|6125.13|
|2017-10-31 09:00:00|0.08937042968493204| 6165.0|
|2017-10-31 10:00:00| 0.1026717642680184|6170.77|
|2017-10-31 11:00:00|0.12814641170317434|6233.74|
|2017-10-31 12:00:00|0.10549754665746006|6201.03|
|2017-10-31 13:00:00|  0.138457897343971|6332.34|
|2017-10-31 14:00:00|0.11779438208175155|6363.13|
|2017-10-31 15:00:00|0.09131997304766154|6365.16|
|2017-10-31 16:00:00|0.09301943334583278|6364.78|
|2017-10-31 17:00:00|0.09206832917391784|6361.79|
|2017-10-31 18:00:00|0.10691324426615843|6335.64|
|2017-10-31 19:00:00|0.12217075088870975|6341.15|
|2017-10-31 20:00:00|0.12817142761147934|6370.08|
|2017-10-31 21:00:00|0.09238581351736848|6393.26|
